In [3]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query
import preprocesser
import datetime
import pandas.tseries.offsets as of  # for timeoffset
from pandas import Series

In [4]:
#uplist[uplist['shcode']=='000080']
#uplist[uplist['shcode']=='000080'].groupby(['shcode']).count()
#tmlist[tmlist['shcode']=='000080']
#tmlist[tmlist['shcode']=='000080'].groupby(['shcode']).count()
#df_bfaf2 = df_bfaf[['index','shcode']].drop_duplicates()

In [54]:
#1.종목명
shcode_list = query.query_shcode_list_today()

#2.target from csvs
df = pd.read_csv("high_ratio_0.05_volume_money_20000000000_v4.csv", 
                 dtype ={'shcode':str})
del df['Unnamed: 0']

In [ ]:
uplist = query.query_up_distinct()
tmlist = query.query_tm_distinct()

In [5]:
#3-1 분리
s = df['index'].str.split(' ').apply(Series,1)#.fillna(0)
df = pd.merge(df, s,left_index =True, right_index =True ,how = 'left')
df['total_n'] = s.groupby(s.index).count().sum(axis=1)
del df['index']
df = df.fillna(0)
df = pd.merge(df, uplist[['shcode','upcode', 'upname']], how = 'left', on = ['shcode'])
df = pd.merge(df, tmlist[['shcode','tmcode', 'tmname']], how = 'left', on = ['shcode'])
df = pd.merge(df, shcode_list, how = 'left', on = ['shcode'])

In [55]:
#3-2 분리
t = df['index'].str.split(' ').apply(Series,1).stack()
t.index = t.index.droplevel(-1)
t.name = 'target_day'
del df['index']
df = df.join(t)
df = pd.merge(df, uplist[['shcode','upcode', 'upname']], how = 'left', on = ['shcode'])
df = pd.merge(df, tmlist[['shcode','tmcode', 'tmname']], how = 'left', on = ['shcode'])
df = pd.merge(df, shcode_list, how = 'left', on = ['shcode'])

In [63]:
df.columns

Index(['shcode', 'target_day', 'upcode', 'upname', 'tmcode', 'tmname',
       'shname'],
      dtype='object')

In [62]:
df.groupby(['upcode','upname'])['shcode'].count().sort_values(ascending= False)

upcode  upname     
308.0   유       통      67
315.0   음식료 담배         64
324.0   전 기/전 자        62
16.0    유   통  업       59
320.0   제       약      58
326.0   운 송/부 품        57
303.0   기타 서비스         55
319.0   화       학      54
331.0   디   지  털       52
323.0   기 계 장 비        50
304.0   IT   종  합      47
19.0    운 수 창 고        42
334.0   통 신 장 비        42
12.0    기       계      41
301.0   코스닥 종합         41
325.0   의 료/정 밀        38
9.0     의   약  품       36
338.0   오락/문화          36
5.0     음 식 료 업        36
337.0   IT   부  품      36
311.0   금       융      34
313.0   IT   S / W     34
345.0   기술성장 기업        33
11.0    철 강 금 속        32
336.0   반   도  체       31
318.0   출 판 매 체        31
307.0   건       설      31
4.0     소   형  주       29
8.0     화       학      29
15.0    운 수 장 비        28
                       ..
10.0    비금속 광물         21
6.0     섬 유 의 복        21
13.0    전 기 전 자        20
18.0    건   설  업       20
314.0   IT   H / W     19
306.0   제       조      19
335.0   정 보 기 기   

In [60]:
df[df['upcode']==308].groupby(['upcode','upname'])['target_day'].min().sort_values(ascending= False)

upcode  upname   
308.0   유       통    20181210
Name: target_day, dtype: object

In [61]:
df[df['upcode']==308].groupby(['upcode','upname'])['target_day'].max().sort_values(ascending= False)

upcode  upname   
308.0   유       통    20191204
Name: target_day, dtype: object

In [19]:
df.head()

,shcode,0,1,2,3,4,5,6,7,8,9,10,total_n,upcode,upname,tmcode,tmname,shname
0,000080,20190424,0,0,0,0,0,0,0,0,0,0,1,1.0,종 합,421.0,日 불매운동 관련주,하이트진로
1,000140,20190708,20190729,0,0,0,0,0,0,0,0,0,2,1.0,종 합,421.0,日 불매운동 관련주,하이트진로홀딩스
2,000150,20190416,20190726,0,0,0,0,0,0,0,0,0,2,1.0,종 합,111.0,지주사,두산
3,000230,20191106,0,0,0,0,0,0,0,0,0,0,1,1.0,종 합,111.0,지주사,일동홀딩스
4,000300,20190510,0,0,0,0,0,0,0,0,0,0,1,1.0,종 합,242.0,탄소나노튜브(CNT),대유플러스


In [6]:
df.columns

Index([ 'shcode',         0,         1,         2,         3,         4,
               5,         6,         7,         8,         9,        10,
       'total_n',  'upcode',  'upname',  'tmcode',  'tmname',  'shname'],
      dtype='object')

In [18]:
df.groupby(['upcode','upname'])['total_n'].count().sort_values(ascending= False)

upcode  upname     
323.0   기 계 장 비        27
326.0   운 송/부 품        24
304.0   IT   종  합      23
319.0   화       학      22
320.0   제       약      22
324.0   전 기/전 자        22
308.0   유       통      21
315.0   음식료 담배         19
303.0   기타 서비스         19
338.0   오락/문화          19
301.0   코스닥 종합         19
16.0    유   통  업       19
325.0   의 료/정 밀        18
331.0   디   지  털       18
311.0   금       융      16
12.0    기       계      16
313.0   IT   S / W     15
345.0   기술성장 기업        15
334.0   통 신 장 비        15
337.0   IT   부  품      14
19.0    운 수 창 고        14
9.0     의   약  품       14
18.0    건   설  업       12
11.0    철 강 금 속        12
332.0   소프트 웨어         12
3.0     중   형  주       11
5.0     음 식 료 업        11
26.0    서 비 스 업        11
307.0   건       설      11
13.0    전 기 전 자        11
                       ..
314.0   IT   H / W      9
316.0   섬 유/의 류         9
318.0   출 판 매 체         9
1.0     종       합       8
322.0   금       속       7
4.0     소   형  주        7
335.0   정 보 기 기   

In [28]:
df[df['tmcode']==386]

,shcode,0,1,2,3,4,5,6,7,8,9,10,total_n,upcode,upname,tmcode,tmname,shname
7,000430,20190212,20190226,0,0,0,0,0,0,0,0,0,2,1.0,종 합,386.0,수소차,대원강업
115,009320,20181212,20181213,20190118,20190121,20190320,20190617,20190718,20190924,0,0,0,8,15.0,운 수 장 비,386.0,수소차,대우부품
121,009680,20190118,20190313,0,0,0,0,0,0,0,0,0,2,15.0,운 수 장 비,386.0,수소차,모토닉
132,010770,20190118,20190618,0,0,0,0,0,0,0,0,0,2,15.0,운 수 장 비,386.0,수소차,평화홀딩스
156,013870,20181219,0,0,0,0,0,0,0,0,0,0,1,15.0,운 수 장 비,386.0,수소차,지엠비코리아
164,014910,20190114,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,386.0,수소차,성문전자
183,023800,20181218,20190121,0,0,0,0,0,0,0,0,0,2,NaN,NaN,386.0,수소차,인지컨트롤스
225,071970,20191113,20191115,0,0,0,0,0,0,0,0,0,2,12.0,기 계,386.0,수소차,STX중공업
233,081000,20181212,20181219,20190125,20190617,20190718,0,0,0,0,0,0,5,NaN,NaN,386.0,수소차,일진다이아
236,085310,20181219,20190129,0,0,0,0,0,0,0,0,0,2,12.0,기 계,386.0,수소차,엔케이


In [21]:
df.groupby(['tmcode','tmname'])['total_n'].count().sort_values(ascending= False)

tmcode  tmname            
386.0   수소차                   33
110.0   화장품                   27
417.0   아프리카 돼지열병(ASF)        26
27.0    자동차부품                 24
421.0   日 불매운동 관련주            20
377.0   치매 관련주                19
373.0   5G(5세대 이동통신)          18
375.0   4차산업 수혜주              17
393.0   갤럭시 부품주               16
250.0   철도                    15
128.0   엔터테인먼트                15
422.0   日 수출 규제 관련주(국산화 등)    15
387.0   블록체인                  15
389.0   면역항암제                 14
175.0   의료기기                  13
398.0   DMZ 평화공원              13
265.0   모바일게임(스마트폰)           13
334.0   사물인터넷                 12
113.0   음식료업종                 12
392.0   스마트팩토리(스마트공장)         11
352.0   가상현실(VR)              11
111.0   지주사                   11
335.0   재난/안전(지진 등)           11
311.0   고령화 사회(노인복지)          11
415.0   그래핀                   11
376.0   유전자 치료제/분석            11
414.0   2019 상반기 신규상장         11
172.0   제약업체                  11
399.0   북한 광물자원개발             10
322.0   건설 중소형  

In [34]:
df.columns[0]

'shcode'

In [42]:
li = [x for x in range(0,12)]
df.columns.isin(li)

In [46]:
df2 = df.copy()

In [47]:
for i in df.columns[df.columns.isin(li)]:
    df2[i] = df2[i].str.slice(stop = 6) 

In [48]:
df2.groupby(['shcode'])

,shcode,0,1,2,3,4,5,6,7,8,9,10,total_n,upcode,upname,tmcode,tmname,shname
0,000080,201904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,종 합,421.0,日 불매운동 관련주,하이트진로
1,000140,201907,201907,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1.0,종 합,421.0,日 불매운동 관련주,하이트진로홀딩스
2,000150,201904,201907,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1.0,종 합,111.0,지주사,두산
3,000230,201911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,종 합,111.0,지주사,일동홀딩스
4,000300,201905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,종 합,242.0,탄소나노튜브(CNT),대유플러스


In [29]:
for i in df.columns:
    print(i)
#fin_result['target_day'].str.slice(stop = 6)

shcode
0
1
2
3
4
5
6
7
8
9
10
total_n
upcode
upname
tmcode
tmname
shname


In [9]:
#1.종목명
shcode_list = query.query_shcode_list_today()

#2.target from csvs
df = pd.read_csv("high_ratio_0.05_volume_money_20000000000_v4.csv", 
                 dtype ={'shcode':str})

#3. target index 추출
fin_result = pd.DataFrame([],columns = ['target_day','shcode'])

for i,v in df['index'].items():
    shcode = df['shcode'].iloc[i].zfill(6)
    target_date = v.split(" ")
    
    target_day_list = pd.to_datetime(target_date, format = "%Y-%m-%d")
    interval_end_timestamp = target_day_list + datetime.timedelta(weeks=6)
    result = pd.DataFrame(dict(target_day = target_day_list.strftime("%Y%m%d"),
                              interval_end = interval_end_timestamp.strftime("%Y%m%d"))).reset_index()
    del result['index']
    result['shcode'] = shcode
    fin_result = fin_result.append(result)
    
#4. make target month
fin_result['target_m'] = fin_result['target_day'].str.slice(stop = 6)
fin_result = pd.merge(fin_result, shcode_list, how = 'inner', on = 'shcode')

#5. target day 확인
target_day_list = pd.to_datetime(fin_result['target_day'], format = "%Y-%m-%d")

#(1) for jupo
#interval calculate before 12 month 
init_jupo_timestamp = target_day_list + of.MonthEnd(n= -12)
end_jupo_timestamp = target_day_list
fin_result['init_jupo'] = init_jupo_timestamp.apply(lambda x: x.strftime("%Y%m%d"))
fin_result['end_jupo'] = end_jupo_timestamp.apply(lambda x: x.strftime("%Y%m%d"))


fin_result = fin_result[fin_result['target_m']>='201909'].reset_index()
del fin_result['index']
fin_result['index'] = fin_result.index

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [ ]:
pd.merge(df_bfaf2, uplist[['shcode','upcode', 'upname', 'per']], how = 'left', on = ['shcode'])

In [10]:
fin_result.iloc[410:415]

,interval_end,shcode,target_day,target_m,shname,init_jupo,end_jupo,index
410,20191105,078590,20190924,201909,두올산업,20180930,20190924,410
411,20191113,078590,20191002,201910,두올산업,20181031,20191002,411
412,20191216,078860,20191104,201911,아이오케이,20181130,20191104,412
413,20191106,079950,20190925,201909,인베니아,20180930,20190925,413
414,20191227,079960,20191115,201911,동양이엔피,20181130,20191115,414


In [29]:
fin_result[fin_result['shname'] == '켐트로닉스']

,interval_end,shcode,target_day,target_m,shname,init_jupo,end_jupo,index
439,20191227,089010,20191115,201911,켐트로닉스,20181130,20191115,439


In [11]:
fin_result.columns

Index(['interval_end', 'shcode', 'target_day', 'target_m', 'shname',
       'init_jupo', 'end_jupo', 'index'],
      dtype='object')

In [57]:
fin_result['init_jupo'].iloc[439], 

'20181130'

In [73]:
for i in 
    # for jupo query
    a = query.query_jupo_vol(fin_result['shcode'].iloc[439], fr='20181130', to='20191130')
    # cumsum index
    c = a.groupby(['shcode','date']).sum().cumsum()
    # multi index to single index
    c = c.reset_index(level=['shcode','date'])
    # calculate max volume
    c_for_value = c[['samo_vol', 'sec_vol', 'ins_vol', 'tusin_vol', 'bank_vol',
                     'jong_vol', 'fund_vol', 'etcom_vol', 'per_vol', 'for_reg_vol',
                     'for_noreg_vol', 'nat_no_vol', 'com_vol', 'for_vol', 'etc_vol']].tail(1)
    c_for_value.sum(axis=1).iloc[0]

1403282

In [74]:
# for jupo query
a = query.query_jupo_vol(fin_result['shcode'].iloc[439], fr=fin_result['init_jupo'].iloc[439], to='20191231')
# cumsum index
c = a.groupby(['shcode','date']).sum().cumsum()
# multi index to single index
c = c.reset_index(level=['shcode','date'])
# calculate max volume
c_for_value = c[['samo_vol', 'sec_vol', 'ins_vol', 'tusin_vol', 'bank_vol',
                 'jong_vol', 'fund_vol', 'etcom_vol', 'per_vol', 'for_reg_vol',
                 'for_noreg_vol', 'nat_no_vol', 'com_vol', 'for_vol', 'etc_vol']]
c['max_vol'] = c_for_value.max(axis=1)
c['max_jupo'] = c_for_value.idxmax(axis=1)
c = c.assign(max3_sum=np.sum(np.partition(c_for_value.values, -3)[:, -3:], 1))



In [40]:
c.tail()

,shcode,date,samo_vol,sec_vol,ins_vol,tusin_vol,bank_vol,jong_vol,fund_vol,etcom_vol,per_vol,for_reg_vol,for_noreg_vol,nat_no_vol,com_vol,for_vol,etc_vol,max_vol,max_jupo,max3_sum
241,089010,20191125,-710041,-32337,164726,628294,7840,-152048,342360,-12019,-1443628,1204700,2153,0,248794,1206853,-12019,1206853,for_vol,3039847
242,089010,20191126,-726422,-31132,160854,626985,7840,-152048,346014,-11319,-1416555,1193828,1955,0,232091,1195783,-11319,1195783,for_vol,3016596
243,089010,20191127,-782848,-33649,155593,616016,7840,-152048,347014,-13841,-1398161,1252257,1827,0,157918,1254084,-13841,1254084,for_vol,3122357
244,089010,20191128,-791287,-57755,157371,601612,7840,-152048,348599,-3295,-1392433,1279569,1827,0,114332,1281396,-3295,1281396,for_vol,3162577
245,089010,20191129,-757529,-83176,157171,620899,7840,-152048,293159,6385,-1403282,1308754,1827,0,86316,1310581,6385,1310581,for_vol,3240234


In [75]:
c.to_excel("1234.xlsx",encoding = 'ms949')

In [76]:
a = query.query_jupo_vol(fin_result['shcode'].iloc[439], fr=fin_result['init_jupo'].iloc[439], to='20191231')
a.to_excel("1234-1.xlsx",encoding = 'ms949')

In [37]:
b = query.query_finance(fin_result['shcode'].iloc[439])

In [39]:
b.columns

Index(['date', 'shcode', 'close', 'volume', 'volume_money', 'high_52',
       'high_52_day', 'low_52', 'low_52_day', 'rotate_ratio', 'burn_ratio',
       'per', 'total_vol', 'ev_ratio', 'vol_scale', 'market_value'],
      dtype='object')

In [38]:
b['total_vol'].astype(int) * 1000

0    13905000
Name: total_vol, dtype: int32

In [ ]:
13905000 - 4056934

In [56]:
c_sam2 = c[['samo_vol', 'sec_vol', 'ins_vol', 'tusin_vol', 'bank_vol',
           'jong_vol', 'fund_vol', 'etcom_vol', 'per_vol', 'for_reg_vol',
           'for_noreg_vol', 'nat_no_vol', 'com_vol', 'for_vol', 'etc_vol']]
c_sam2.sum(axis=1)

0          768
1         1391
2         4713
3         8197
4         5364
5         5952
6         9889
7         3784
8         4281
9         6685
10        5017
11      -12412
12      -20046
13      -21852
14      -31687
15      -28676
16      -36482
17      -40789
18      -44662
19       14222
20       62999
21       61665
22       68318
23       70326
24       60426
25       55683
26       52569
27       40983
28       34815
29       23644
        ...   
216     209352
217     341543
218     271352
219     510918
220     859740
221     884779
222     897740
223     914734
224     947794
225    1002866
226    1002083
227    1030267
228    1133791
229    1090623
230    1146198
231    1153030
232    1296123
233    1334199
234    1313963
235    1468573
236    1475529
237    1455159
238    1392198
239    1440078
240    1506026
241    1443628
242    1416555
243    1398161
244    1392433
245    1403282
Length: 246, dtype: int64

In [46]:
c_sam = c[['samo_vol', 'sec_vol', 'ins_vol', 'tusin_vol', 'bank_vol',
           'jong_vol', 'fund_vol', 'etcom_vol', 'per_vol', 'for_reg_vol',
           'for_noreg_vol', 'nat_no_vol', 'com_vol', 'for_vol', 'etc_vol']].iloc[245]

In [48]:
c_sam

samo_vol         -757529
sec_vol           -83176
ins_vol           157171
tusin_vol         620899
bank_vol            7840
jong_vol         -152048
fund_vol          293159
etcom_vol           6385
per_vol         -1403282
for_reg_vol      1308754
for_noreg_vol       1827
nat_no_vol             0
com_vol            86316
for_vol          1310581
etc_vol             6385
Name: 245, dtype: int64

In [51]:
c_sam.sum(axis=0)

1403282

In [41]:
# 외인
1310581 / 13905000

0.09425249910104279

In [ ]:
1403282 / 13905000

In [44]:
# 투신
3240234 / 13905000

0.23302653721682848

In [65]:
13905000 - 4056939

9848061

In [67]:
1098523 / 9848061

0.11154713603012817

In [16]:
b = query.query_finance(fin_result['shcode'].iloc[414])

In [17]:
b

,date,shcode,close,volume,volume_money,high_52,high_52_day,low_52,low_52_day,rotate_ratio,burn_ratio,per,total_vol,ev_ratio,vol_scale,market_value
0,20191203,079960,17350,145243,2469,19500,20191119,9000,20190107,15.95,1.85,22.73,7860,40,00001,1348


In [35]:
query.query_finance(fin_result['shcode'].iloc[439])

,date,shcode,close,volume,volume_money,high_52,high_52_day,low_52,low_52_day,rotate_ratio,burn_ratio,per,total_vol,ev_ratio,vol_scale,market_value
0,20191203,089010,16300,440065,7079,18500,20191119,5230,20181226,13.93,3.16,103.49,13905,40,00001,2225


In [23]:
type(b['total_vol'].iloc[0])

str

In [25]:
c.iloc[235]

shcode             079960
date             20191115
samo_vol            48992
sec_vol             99560
ins_vol              1584
tusin_vol           31333
bank_vol           -90339
jong_vol             1224
fund_vol            -1584
etcom_vol           21943
per_vol            229600
for_reg_vol       -329458
for_noreg_vol      -12855
nat_no_vol              0
com_vol             90770
for_vol           -342313
etc_vol             21943
max_vol            229600
max_jupo          per_vol
max3_sum           419930
Name: 235, dtype: object

In [28]:
419930 / 7860000

0.05342620865139949

In [24]:
b['total_vol'].astype(int) * 1000

0    7860000
Name: total_vol, dtype: int32

In [ ]:
'최고가_52','최고가일_52','최저가_52','최저가일_52','소진율','회전율','PER'
'상장주식수_천','증거금율','수량단위',